# Site Dev

# Site App with Database 

* About Me Section
* Notepad: Create, Read
* Counter

## Create a database server using Docker

### Postgres Docker

In [ ]:
# Without volume/data connected
# !docker run --name postgres-sql -e POSTGRES_USER=matt -e POSTGRES_PASSWORD=password  -d postgres

In [ ]:
# With volume/data connected
!docker run -d \
    --name postgres-sql \
    -e POSTGRES_USER=matt \
    -e POSTGRES_PASSWORD=password \
    -v $(pwd)/data/:/var/lib/postgresql/data/ \
    -p 5000:5432 \
    postgres

In [ ]:
!docker ps

In [ ]:
!ls -a

In [ ]:
# Check the data
# In a terminal
# sudo su
# cd $(pwd)/data/
# ls -a

In [ ]:
# Test the data of the DB
# docker rm -f <CONTAINER_ID>
# run the docker run <...>
# check the pgAdmin
# the data persists

### pgAdmin

In [ ]:
!docker run -p 8000:80 \
    -e 'PGADMIN_DEFAULT_EMAIL=matt@example.com' \
    -e 'PGADMIN_DEFAULT_PASSWORD=password' \
    -d dpage/pgadmin4

In [ ]:
!docker ps

### Create a firewall (GCP)

In [ ]:
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000,tcp:8000,tcp:8501,tcp:9000 --source-ranges=0.0.0.0/0

### Open pgAdmin Browser

In [ ]:
# http://<IP_ADDRESS>:<PORT>
# Login with credentials

# Register Server
# HOST: <IP_ADDRESS>
# PORT: 5000
# USER: <POSTGRES_USER>
# PASSWORD: <PASSWORD>

## Install and import libraries

In [ ]:
!pip install streamlit psycopg2-binary -q

In [1]:
# Restart the kernel

In [2]:
import streamlit as st
import psycopg2

In [3]:
!mkdir app

In [4]:
cd app

/home/jupyter/tech-stack/13-App-site-blog/site-/app


In [5]:
%%writefile env.py
# DB Credentials
DBNAME="matt"
USER='matt'
HOST='34.173.135.93'
PORT='5000'
PASSWORD='password'

Writing env.py


In [42]:
%%writefile app.py
# Import libraries
import streamlit as st
import psycopg2
import os
import time
from env import *

#----------Page Configuration----------# 
st.set_page_config(page_title="Matt Cloud Tech",
                   page_icon=":cloud:",
                   menu_items={
                       'About':"# Matt Cloud Tech"})
#----------About Me Section----------#
st.title(":cloud: Matt Cloud Tech")
st.subheader("", divider="rainbow")

st.write("""
        ### Good day :wave:.
        ### My name is :blue[Matt]. I am a Cloud Technology Enthusiast.
        ### Currently, I am learning and building Cloud Infrastructure, Data and CI/CD Pipelines, and Intelligent Systems. 
        """)

st.divider()
st.write(":link: :computer: [Personal Website](https://)")
st.write(":link: :book: [Project Repository](https://)")
st.write(":link: :notebook: [Blog](https://)")
st.write(":link: :hand: [Connect with me](https://)")

#----------Notepad----------#
# Notepad Section
# Header
st.header("Notepad :notebook:",divider="rainbow")
st.caption("""
            Add your thoughts here! It will be stored in a database! \n
            :warning: :red[Do not add sensitive data.]
            """)
# Variable
database_name = DBNAME

con = psycopg2.connect(f"""
                       dbname={DBNAME}
                       user={USER}
                       host={HOST}
                       port={PORT}
                       password={PASSWORD}
                       """)
cur = con.cursor()
# Create a table if not exists
cur.execute("CREATE TABLE IF NOT EXISTS notes(id serial PRIMARY KEY, name varchar, header varchar, note varchar, time varchar)")
con.commit()

# Inputs
name = st.text_input("Your Name")
header = st.text_input("Header")
note = st.text_area("Note")
if st.button("Add a note"):
    time = time.strftime("Date: %Y-%m-%d | Time: %H:%M:%S UTC")
    st.write(f""" \n
            ##### :pencil: {header} \n
            #### {note} \n
            :man: {name} \n
            :watch: {time}""")
    st.success("Successfully Added.")
    # st.balloons()
    ### Insert into adatabase
    SQL = "INSERT INTO notes (name, header, note, time) VALUES(%s, %s, %s, %s);"
    data = (name, header, note, time)
    cur.execute(SQL, data)
    con.commit()
    
# Previous Notes 
st.divider()
st.write("### *Previous Notes*")
# Write the data
cur.execute("""
            SELECT * 
            FROM notes
            ORDER BY time DESC
            """)
for id, name, header, note, time in cur.fetchall():
    st.write(f""" \n
            ##### :pencil: {header} \n
            #### {note} \n
            :man: {name} \n
            :watch: {time}""")

    with st.expander(f"Edit or Delete (ID #: {id})"):
        name = st.text_input(f"Your Name (ID #: {id})", name)
        header = st.text_input(f"Header (ID #: {id})", header)
        note = st.text_area(f"Note (ID #: {id})", note)
        if st.button(f"UPDATE ID #: {id}"):
            SQL = " UPDATE notes SET id=%s, name=%s, header=%s, note=%s WHERE id = %s"
            data = (id, name, header, note, id)
            cur.execute(SQL, data)
            con.commit()
            st.success("Successfully Edited.")
        if st.button(f"DELETE ID #: {id}"):
            cur.execute(f"DELETE FROM notes WHERE id = {id}")
            con.commit()
            st.success("Successfully Deleted.")
    st.subheader("",divider="gray")
    
# Close Connection
cur.close()
con.close()

#----------Counter----------#
# Title
st.header("Counter")
st.caption("""
            Count every request in this app.
            """)
st.subheader("",divider="rainbow")

# Variable
database_name = DBNAME

con = psycopg2.connect(f"""
                       dbname={DBNAME}
                       user={USER}
                       host={HOST}
                       port={PORT}
                       password={PASSWORD}
                       """)
cur = con.cursor()
# Create a table if not exists
cur.execute("CREATE TABLE IF NOT EXISTS counter(id serial PRIMARY KEY, view int, time varchar)")
con.commit()

# Counter
import time
time = time.strftime("Date: %Y-%m-%d | Time: %H:%M:%S UTC")
view = 1
### Insert into a database
SQL = "INSERT INTO counter (view, time) VALUES(%s, %s);"
data = (view, time)
cur.execute(SQL, data)
con.commit()

# Total views
cur.execute("""
                SELECT SUM(view) 
                FROM counter
                """)
st.write(f"### Total views: **{cur.fetchone()[0]}**")

# Current view
st.write(f"Current: {time}")
# Previous hits
st.divider()
with st.expander("See Previous Views"):
    st.write("**Previous Views**")
    # Write the data
    cur.execute("""
                SELECT * 
                FROM counter
                ORDER BY time DESC
                """)
    for _, _, time in cur.fetchall():
        st.text(f"{time}")
    
# Close Connection
cur.close()
con.close()

Overwriting app.py


In [7]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit psycopg2-binary -q
COPY app.py env.py ./app/
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0"]

Writing Dockerfile


In [ ]:
# Build an image

In [8]:
!docker build -t app . -q

sha256:b53995952b15554a05037db570689177a35696f739d36117fd748b4a1cb9ed16


In [9]:
# List images
!docker images

REPOSITORY       TAG       IMAGE ID       CREATED                  SIZE
app              latest    b53995952b15   Less than a second ago   1.5GB
dpage/pgadmin4   latest    f70baa3facc1   2 weeks ago              534MB
python           latest    2acccf902fa3   3 weeks ago              1.02GB
postgres         latest    fbd1be2cbb1f   7 weeks ago              417MB


In [10]:
# Run the app in detached mode
!docker run -d -p 8501:8501 -v $(pwd):/app app

c71b041ff6d18588d498cbb0343810210ce1d94d9e64015333d517839921ffe5


In [11]:
!docker ps -a

CONTAINER ID   IMAGE            COMMAND                  CREATED                  STATUS                  PORTS                                            NAMES
c71b041ff6d1   app              "streamlit run app.p…"   Less than a second ago   Up Less than a second   0.0.0.0:8501->8501/tcp, :::8501->8501/tcp        modest_wu
9a0dcd8c2bc4   dpage/pgadmin4   "/entrypoint.sh"         7 minutes ago            Up 7 minutes            443/tcp, 0.0.0.0:8000->80/tcp, :::8000->80/tcp   blissful_goodall
4d1deb153fbe   postgres         "docker-entrypoint.s…"   7 minutes ago            Up 7 minutes            0.0.0.0:5000->5432/tcp, :::5000->5432/tcp        postgres-sql


In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

#### Connect to the database from Jupyter Lab

In [ ]:
# Import libraries
import psycopg2
import os
import time
from env import *

# Variable
database_name = DBNAME

con = psycopg2.connect(f"""
                       dbname={DBNAME}
                       user={USER}
                       host={HOST}
                       port={PORT}
                       password={PASSWORD}
                       """)
cur = con.cursor()

In [ ]:
cur = con.cursor()
SQL = "SELECT * FROM notes"
cur.execute(SQL)
cur.fetchall()

In [ ]:
dir(st)

In [ ]:
id = 46
name='Matt'
header='Hello'
note='Hi'

In [ ]:
# Update
cur.execute(f"UPDATE notes SET id={id}, name='{name}', header='{header}', note='{note}' WHERE id = {id}")
con.commit()